In [40]:
import nibabel as nib
import numpy as np
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter
from plotly.subplots import make_subplots

In [41]:
def load_and_preprocess_nifti(nifti_path):
    print(f"Loading NIFTI file from {nifti_path}")
    nifti_img = nib.load(nifti_path)
    nifti_data = nifti_img.get_fdata()
    print("Normalizing data")
    normalized_data = (nifti_data - np.min(nifti_data)) / (np.max(nifti_data) - np.min(nifti_data))
    print("Data loaded and preprocessed")
    return normalized_data

In [42]:
def save_as_nrrd(data, file_path):
    """Save the numpy array as NRRD file if needed."""
    nrrd.write(file_path, data)

def downsample_3d(data, factor):
    """Downsample 3D data by a factor"""
    return data[::factor, ::factor, ::factor]

# Numpy Implementation

In [43]:
def compute_hog3d(ct_data, block_size=8, cell_size=4, stride=4, num_bins=12):
    print("Starting HOG3D computation")

    # Apply Gaussian smoothing to the data
    # Sigma value is set to 1, increasing it will result in more smoothing
    smoothed_data = gaussian_filter(ct_data, sigma=1)

    # Compute the gradient of the smoothed data
    gx, gy, gz = np.gradient(smoothed_data)

    # Compute the magnitude, phi and theta values
    magnitude = np.sqrt(gx**2 + gy**2 + gz**2)

    # Phi is the angle between the x-axis and the projection of the gradient vector onto the xy-plane
    phi = np.arctan2(np.sqrt(gx**2 + gy**2), gz)

    # Theta is the angle between the z-axis and the gradient vector
    theta = np.arctan2(gy, gx)

    # Initialize the lists to store the HOG3D features, positions and orientations
    hog3d_features = []
    hog3d_positions = []
    hog3d_orientations = []

    # Calculate total blocks and initialize the current block counter
    total_blocks = ((ct_data.shape[0] - block_size) // stride + 1) * \
                   ((ct_data.shape[1] - block_size) // stride + 1) * \
                   ((ct_data.shape[2] - block_size) // stride + 1)
    current_block = 0



    for x in range(0, ct_data.shape[0] - block_size + 1, stride): # Loop over the x-axis
        for y in range(0, ct_data.shape[1] - block_size + 1, stride): # Loop over the y-axis
            for z in range(0, ct_data.shape[2] - block_size + 1, stride): # Loop over the z-axis
                block_hist = np.zeros((2 * num_bins, (block_size // cell_size) ** 3)) # Initialize the block histogram

                cell_index = 0
                for i in range(0, block_size, cell_size): # Loop over the x-axis of the block
                    for j in range(0, block_size, cell_size): # Loop over the y-axis of the block
                        for k in range(0, block_size, cell_size): # Loop over the z-axis of the block

                            # Extract the cell from the phi, theta and magnitude arrays
                            cell_phi = phi[x+i:x+i+cell_size, y+j:y+j+cell_size, z+k:z+k+cell_size] 
                            cell_theta = theta[x+i:x+i+cell_size, y+j:y+j+cell_size, z+k:z+k+cell_size]
                            cell_magnitude = magnitude[x+i:x+i+cell_size, y+j:y+j+cell_size, z+k:z+k+cell_size]

                            # 1D histogram for phi angles
                            hist_phi, _ = np.histogram(
                                cell_phi.ravel(),
                                bins=num_bins,
                                range=(-np.pi, np.pi),
                                weights=cell_magnitude.ravel()
                            )

                            # 1D histogram for theta angles
                            hist_theta, _ = np.histogram(
                                cell_theta.ravel(),
                                bins=num_bins,
                                range=(-np.pi, np.pi),
                                weights=cell_magnitude.ravel()
                            )

                            # Concatenate the histograms for phi and theta
                            block_hist[:, cell_index] = np.concatenate([hist_phi, hist_theta])
                            cell_index += 1

                block_hist = block_hist.ravel()  # Reshape the block_hist array
                block_hist /= np.linalg.norm(block_hist) + 1e-5  # Normalize the histogram

                hog3d_features.append(block_hist)                                                           # Append the block histogram to the HOG3D features list
                hog3d_positions.append([x + block_size // 2, y + block_size // 2, z + block_size // 2])     # Append the block position to the HOG3D positions list
                hog3d_orientations.append([np.mean(cell_phi), np.mean(cell_theta)])                         # Append the block orientation to the HOG3D orientations list

                # statements to keep track of the progress
                current_block += 1
                if current_block % 10000 == 0 or current_block == total_blocks:
                    print(f"Processed {current_block}/{total_blocks} blocks")

    print("HOG3D computation completed")
    return np.array(hog3d_features), np.array(hog3d_positions), np.array(hog3d_orientations)


In [44]:
def visualize_hog3d(ct_data, hog3d_features, hog3d_positions, hog3d_orientations, threshold=0.75, save_path=None):
    print("Starting visualization")
    feature_magnitudes = np.linalg.norm(hog3d_features, axis=1)
    
    # Use a more discriminative threshold
    magnitude_threshold = np.percentile(feature_magnitudes, threshold * 100)
    mask = feature_magnitudes > magnitude_threshold
    
    filtered_positions = hog3d_positions[mask]
    filtered_magnitudes = feature_magnitudes[mask]
    filtered_orientations = hog3d_orientations[mask]
    
    # Apply non-linear transformation to magnitudes
    filtered_magnitudes = np.power(filtered_magnitudes, 0.3)  # Adjust the power for desired spread
    
    # Find the bounding box of important features
    min_x, min_y, min_z = np.min(filtered_positions, axis=0)
    max_x, max_y, max_z = np.max(filtered_positions, axis=0)
    
    # Add some padding to the bounding box
    padding = 10
    min_x, min_y, min_z = max(0, min_x - padding), max(0, min_y - padding), max(0, min_z - padding)
    max_x, max_y, max_z = min(ct_data.shape[0], max_x + padding), min(ct_data.shape[1], max_y + padding), min(ct_data.shape[2], max_z + padding)
    
    # Create subplots
    fig = make_subplots(
        rows=1, cols=3,
        specs=[[{'type': 'scene'}, {'type': 'scene'}, {'type': 'scene'}]],
        subplot_titles=("Feature Positions", "Orientations", "Combined")
    )
    
    # 1. Feature position points
    fig.add_trace(
        go.Scatter3d(
            x=filtered_positions[:, 0],
            y=filtered_positions[:, 1],
            z=filtered_positions[:, 2],
            mode='markers',
            marker=dict(
                size=1,
                color=filtered_magnitudes,
                colorscale='Viridis',
                opacity=0.5,
                colorbar=dict(title="Feature Magnitude", x=0.3)
            ),
            name='HOG3D Features'
        ),
        row=1, col=1
    )
    
    # 2. Orientation cones
    u = np.sin(filtered_orientations[:, 0]) * np.cos(filtered_orientations[:, 1])
    v = np.sin(filtered_orientations[:, 0]) * np.sin(filtered_orientations[:, 1])
    w = np.cos(filtered_orientations[:, 0])
    
    scale_factor = 5
    u *= scale_factor
    v *= scale_factor
    w *= scale_factor
    
    fig.add_trace(
        go.Cone(
            x=filtered_positions[:, 0],
            y=filtered_positions[:, 1],
            z=filtered_positions[:, 2],
            u=u,
            v=v,
            w=w,
            colorscale='Plotly3',
            sizemode="absolute",
            opacity=0.2,
            sizeref=3,
            
        ),
        row=1, col=2
    )
    
    # 3. Combined visualization
    fig.add_trace(
        go.Scatter3d(
            x=filtered_positions[:, 0],
            y=filtered_positions[:, 1],
            z=filtered_positions[:, 2],
            mode='markers',
            marker=dict(
                size=1,
                color=filtered_magnitudes,
                colorscale='Viridis',
                opacity=1,
                
            ),
            name='HOG3D Features'
        ),
        row=1, col=3
    )
    
    fig.add_trace(
        go.Cone(
            x=filtered_positions[:, 0],
            y=filtered_positions[:, 1],
            z=filtered_positions[:, 2],
            u=u,
            v=v,
            w=w,
            colorscale='Plotly3',
            sizemode="absolute",
            opacity=0.2,
            sizeref=5,
            
        ),
        row=1, col=3
    )
    
    # Update layout for all subplots
    for i in range(1, 4):
        fig.update_scenes(
            xaxis=dict(title="X", range=[min_x, max_x]),
            yaxis=dict(title="Y", range=[min_y, max_y]),
            zaxis=dict(title="Z", range=[min_z, max_z]),
            aspectmode='data',
            row=1, col=i
        )
    
    fig.update_layout(
        width=5000,
        height=1000,
        title="HOG3D Feature and Orientation Visualization (Cropped)"
    )
    
    fig.show()
    
    if save_path:
        fig.write_html(save_path)
        print(f"Visualization saved to {save_path}")
    
    print("Visualization completed")

In [45]:
nifti_file = "NIFTY/CVD0226_Image.nii.gz" # Replace with the correct file path
ct_data = load_and_preprocess_nifti(nifti_file)
print("Processed data shape:", ct_data.shape)

downsample_factor = 2
downsampled_ct_data = downsample_3d(ct_data, downsample_factor)
print("Downsampled data shape:", downsampled_ct_data.shape)

# Calculate expected shapes
"""
Block Size: Increase to capture more information in each block
Cell Size: Increase to capture more information in each cell
Stride: Increase to reduce the number of blocks
Num Bins: Increase to capture more information in each histogram.
"""
block_size = 4
cell_size = 2
stride = 2
num_bins = 18

num_blocks_x = (downsampled_ct_data.shape[0] - block_size) // stride + 1
num_blocks_y = (downsampled_ct_data.shape[1] - block_size) // stride + 1
num_blocks_z = (downsampled_ct_data.shape[2] - block_size) // stride + 1
total_blocks = num_blocks_x * num_blocks_y * num_blocks_z

features_per_block = (2 * num_bins) * ((block_size // cell_size) ** 3)


# Compute HOG3D
hog3d_features, hog3d_positions, hog3d_orientations = compute_hog3d(downsampled_ct_data, block_size=block_size, cell_size=cell_size, stride=stride, num_bins=num_bins)

# Print expected and computed shapes alternately
print("")
print("Downsample Factor:", downsample_factor)
print("Total Blocks: ",(total_blocks))
print(f"Block Size: {block_size}, Cell Size: {cell_size}, Stride: {stride}, Num Bins: {num_bins}")
print("")
print("Expected HOG3D features shape:", (total_blocks, features_per_block))
print("Computed HOG3D features shape:", hog3d_features.shape)
print("")
print("Expected HOG3D positions shape:", (total_blocks, 3))
print("Computed HOG3D positions shape:", hog3d_positions.shape)
print("")
print("Expected HOG3D orientations shape:", (total_blocks, 2))
print("Computed HOG3D orientations shape:", hog3d_orientations.shape)

Loading NIFTI file from NIFTY/CVD0226_Image.nii.gz
Normalizing data
Data loaded and preprocessed
Processed data shape: (512, 512, 557)
Downsampled data shape: (128, 128, 140)
Starting HOG3D computation
Processed 10000/273861 blocks
Processed 20000/273861 blocks
Processed 30000/273861 blocks
Processed 40000/273861 blocks
Processed 50000/273861 blocks
Processed 60000/273861 blocks
Processed 70000/273861 blocks
Processed 80000/273861 blocks
Processed 90000/273861 blocks
Processed 100000/273861 blocks
Processed 110000/273861 blocks
Processed 120000/273861 blocks
Processed 130000/273861 blocks
Processed 140000/273861 blocks
Processed 150000/273861 blocks
Processed 160000/273861 blocks
Processed 170000/273861 blocks
Processed 180000/273861 blocks
Processed 190000/273861 blocks
Processed 200000/273861 blocks
Processed 210000/273861 blocks
Processed 220000/273861 blocks
Processed 230000/273861 blocks
Processed 240000/273861 blocks
Processed 250000/273861 blocks
Processed 260000/273861 blocks
P

In [46]:
# Visualize HOG3D features
save_path = "hog3d_visualization.html"  # Set to None if you don't want to save
# visualize_hog3d(ct_data, hog3d_features, hog3d_positions, hog3d_orientations, threshold=0.2, save_path=save_path)

In [47]:
import numpy as np

# Assuming hog3d_features is a 2D array with shape (num_blocks, num_features)
# Example: hog3d_features = np.random.rand(8843400, 288)

# Calculate min, max, and mean for each block
min_features = np.min(hog3d_features, axis=1)
max_features = np.max(hog3d_features, axis=1)
mean_features = np.mean(hog3d_features, axis=1)

# Stack the results along the second dimension
reduced_features = np.stack((min_features, max_features, mean_features), axis=1)

# Check the shape of the reduced features
print("Original shape:", hog3d_features.shape)
print("Reduced shape:", reduced_features.shape)

Original shape: (273861, 288)
Reduced shape: (273861, 3)


In [48]:
from sklearn.decomposition import PCA

def apply_pca(features, n_components=50):
    """
    Applies PCA to reduce the dimensionality of the input features.

    Args:
        features (np.array): The input feature array.
        n_components (int): The number of principal components to retain.

    Returns:
        np.array: The reduced feature array.
    """
    pca = PCA(n_components=n_components)
    reduced_features = pca.fit_transform(features)
    return reduced_features


In [49]:
reduced_features_pca = apply_pca(hog3d_features, n_components=80)
print("PCA Reduced Features Shape:", reduced_features_pca.shape)

PCA Reduced Features Shape: (273861, 80)


In [50]:
def reduced_visualize(reduced_features, threshold=0.75, save_path=None):
    """
    Visualizes the PCA-reduced HOG3D features using Plotly.

    Parameters:
    - reduced_features: The PCA-reduced features.
    - threshold: Threshold for visualizing features.
    - save_path: Path to save the visualization. If None, displays the plot.
    """
    print("Starting PCA visualization")
    
    # Calculate magnitudes of the reduced features
    feature_magnitudes = np.linalg.norm(reduced_features, axis=1)
    
    # Use a more discriminative threshold
    magnitude_threshold = np.percentile(feature_magnitudes, threshold * 100)
    mask = feature_magnitudes > magnitude_threshold
    
    filtered_reduced_features = reduced_features[mask]
    filtered_magnitudes = feature_magnitudes[mask]
    
    # Apply non-linear transformation to magnitudes for visualization
    filtered_magnitudes = np.power(filtered_magnitudes, 0.3)  # Adjust the power for desired spread
    
    # Create subplots
    fig = make_subplots(
        rows=1, cols=1,
        specs=[[{'type': 'scatter3d'}]],
        subplot_titles=("PCA-Reduced Features")
    )
    
    # Plot the reduced features
    fig.add_trace(
        go.Scatter3d(
            x=filtered_reduced_features[:, 0],
            y=filtered_reduced_features[:, 1],
            z=filtered_reduced_features[:, 2] if filtered_reduced_features.shape[1] > 2 else [0] * len(filtered_reduced_features),
            mode='markers',
            marker=dict(
                size=1,
                color=filtered_magnitudes,
                colorscale='Viridis',
                opacity=0.5,
                colorbar=dict(title="Feature Magnitude")
            ),
            name='PCA-Reduced Features'
        ),
        row=1, col=1
    )
    
    fig.update_layout(
        title="PCA-Reduced Feature Visualization",
        scene=dict(
            xaxis=dict(title="Component 1"),
            yaxis=dict(title="Component 2"),
            zaxis=dict(title="Component 3") if filtered_reduced_features.shape[1] > 2 else dict(title=""),
        ),
        width=1000,
        height=800
    )
    
    fig.show()
    
    if save_path:
        fig.write_html(save_path)
        print(f"Visualization saved to {save_path}")
    
    print("Visualization completed")


In [51]:
# reduced_visualize(reduced_features=reduced_features_pca, threshold=0.75, save_path=None)

In [52]:
def load_and_process_label(label_file, hog3d_positions, downsample_factor):
    print(f"Loading label file: {label_file}")
    label_img = nib.load(label_file)
    label_data = label_img.get_fdata()
    print(f"Original label data shape: {label_data.shape}")
    
    # Adjust positions for downsampling
    adjusted_positions = hog3d_positions * downsample_factor
    print(f"Adjusted positions shape: {adjusted_positions.shape}")
    
    # Extract labels for each HOG3D feature position
    labels = []
    for pos in adjusted_positions:
        x, y, z = map(int, pos)
        # Ensure the position is within the label data bounds
        x = min(max(x, 0), label_data.shape[0] - 1)
        y = min(max(y, 0), label_data.shape[1] - 1)
        z = min(max(z, 0), label_data.shape[2] - 1)
        labels.append(label_data[x, y, z])
    
    labels = np.array(labels)
    print(f"Processed labels shape: {labels.shape}")
    return labels

In [53]:
label_file = "Labels/CVD0226_Label.nii"
labels = load_and_process_label(label_file, hog3d_positions, downsample_factor)
print("Labels shape:", labels.shape)

Loading label file: Labels/CVD0226_Label.nii
Original label data shape: (512, 512, 557)
Adjusted positions shape: (273861, 3)
Processed labels shape: (273861,)
Labels shape: (273861,)


In [54]:
# Ensure labels are integers
labels = labels.astype(int)

# Get unique labels
unique_labels = np.unique(labels)
print("Unique labels:", unique_labels)

# Print label distribution
for label in unique_labels:
    count = np.sum(labels == label)
    percentage = (count / len(labels)) * 100
    print(f"Label {label}: {count} ({percentage:.2f}%)")

Unique labels: [0 1 2 3]
Label 0: 273743 (99.96%)
Label 1: 45 (0.02%)
Label 2: 29 (0.01%)
Label 3: 44 (0.02%)


In [55]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

def train_svm(features, labels, test_size=0.2, random_state=42):
    print("Starting SVM training process...")
    
    # Filter out samples with label 0
    mask = labels != 0
    features = features[mask]
    labels = labels[mask]

    print(f"Total number of samples (excluding label 0): {len(features)}")
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=test_size, random_state=random_state)
    print(f"Training set size: {len(X_train)}")
    print(f"Testing set size: {len(X_test)}")

    # Standardize the features
    print("Scaling features...")
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Train the SVM model
    print("Training the SVM model with RBF kernel...")
    svm_model = SVC(kernel='rbf', C=1.0, random_state=random_state, verbose=True)
    svm_model.fit(X_train_scaled, y_train)

    # Make predictions on the test set
    print("Making predictions on the test set...")
    y_pred = svm_model.predict(X_test_scaled)

    # Display the classification report and confusion matrix
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    return svm_model, scaler


In [56]:
# Train the SVM
svm_model, scaler = train_svm(hog3d_features, labels)

Starting SVM training process...
Total number of samples (excluding label 0): 118
Training set size: 94
Testing set size: 24
Scaling features...
Training the SVM model with RBF kernel...
[LibSVM].*
optimization finished, #iter = 92
obj = -22.181881, rho = -0.136573
nSV = 57, nBSV = 15
.*
optimization finished, #iter = 89
obj = -30.193120, rho = -0.406061
nSV = 68, nBSV = 26
.*
optimization finished, #iter = 85
obj = -25.477519, rho = -0.182463
nSV = 60, nBSV = 18
Total nSV = 94
Making predictions on the test set...

Classification Report:
              precision    recall  f1-score   support

           1       0.69      1.00      0.81        11
           2       1.00      0.20      0.33         5
           3       0.71      0.62      0.67         8

    accuracy                           0.71        24
   macro avg       0.80      0.61      0.60        24
weighted avg       0.76      0.71      0.67        24


Confusion Matrix:
[[11  0  0]
 [ 2  1  2]
 [ 3  0  5]]


In [57]:
# To make predictions on the entire image:
def predict_image(svm_model, scaler, hog3d_features):
    print("Making predictions on the entire image...")
    scaled_features = scaler.transform(hog3d_features)
    predictions = svm_model.predict(scaled_features)
    print(f"Predictions shape: {predictions.shape}")
    return predictions

In [58]:
image_predictions = predict_image(svm_model, scaler, hog3d_features)

# Reshape predictions to match downsampled image dimensions
prediction_shape = (
    (downsampled_ct_data.shape[0] - block_size) // stride + 1,
    (downsampled_ct_data.shape[1] - block_size) // stride + 1,
    (downsampled_ct_data.shape[2] - block_size) // stride + 1
)
reshaped_predictions = image_predictions.reshape(prediction_shape)
print("Reshaped predictions shape:", reshaped_predictions.shape)

Making predictions on the entire image...
Predictions shape: (273861,)
Reshaped predictions shape: (63, 63, 69)


In [59]:
import numpy as np
import nibabel as nib
import plotly.graph_objects as go

# Load the label NIFTI file
nifti_file_path = 'Labels/CVD0226_Label.nii'  # Replace with your actual file path
nifti_img = nib.load(nifti_file_path)
label_data = nifti_img.get_fdata()

# Create a 3D plot using Plotly
fig = go.Figure()

# Extract the indices of the non-zero labels
non_zero_indices = np.nonzero(label_data)
x, y, z = non_zero_indices
colors = label_data[non_zero_indices]

# Add the 3D scatter plot for the labels
fig.add_trace(go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=2,
        color=colors,
        colorscale='Viridis',  # Color scale
        opacity=0.5
    )
))

# Set plot titles and axis labels
fig.update_layout(
    title='3D Scatter Plot of Label Data',
    scene=dict(
        xaxis_title='X Axis',
        yaxis_title='Y Axis',
        zaxis_title='Z Axis'
    )
)

# Display the plot
fig.show()


In [68]:
def test(path):
    # Load the NIFTI file using nibabel and preprocess it
    ct_data = load_and_preprocess_nifti(path)
    print("Processed data shape:", ct_data.shape)

    downsample_factor = 1
    downsampled_ct_data = downsample_3d(ct_data, downsample_factor)
    print("Downsampled data shape:", downsampled_ct_data.shape)

    # Define HOG3D parameters
    block_size = 4
    cell_size = 2
    stride = 2
    num_bins = 18

    num_blocks_x = (downsampled_ct_data.shape[0] - block_size) // stride + 1
    num_blocks_y = (downsampled_ct_data.shape[1] - block_size) // stride + 1
    num_blocks_z = (downsampled_ct_data.shape[2] - block_size) // stride + 1
    total_blocks = num_blocks_x * num_blocks_y * num_blocks_z

    features_per_block = (2 * num_bins) * ((block_size // cell_size) ** 3)

    # Compute HOG3D features
    hog3d_features, hog3d_positions, hog3d_orientations = compute_hog3d(downsampled_ct_data, block_size=block_size, cell_size=cell_size, stride=stride, num_bins=num_bins)

    # Print expected and computed shapes alternately
    print("")
    print("Downsample Factor:", downsample_factor)
    print("Total Blocks:", total_blocks)
    print(f"Block Size: {block_size}, Cell Size: {cell_size}, Stride: {stride}, Num Bins: {num_bins}")
    print("")
    print("Expected HOG3D features shape:", (total_blocks, features_per_block))
    print("Computed HOG3D features shape:", hog3d_features.shape)
    print("")
    print("Expected HOG3D positions shape:", (total_blocks, 3))
    print("Computed HOG3D positions shape:", hog3d_positions.shape)
    print("")
    print("Expected HOG3D orientations shape:", (total_blocks, 2))
    print("Computed HOG3D orientations shape:", hog3d_orientations.shape)

    # Predict image
    predicted_labels = predict_image(svm_model, scaler, hog3d_features)

    # Determine the shape of the prediction output
    prediction_shape = (
        (downsampled_ct_data.shape[0] - block_size) // stride + 1,
        (downsampled_ct_data.shape[1] - block_size) // stride + 1,
        (downsampled_ct_data.shape[2] - block_size) // stride + 1
    )

    # Reshape predictions to match the dimensions of the downsampled image
    reshaped_predictions = predicted_labels.reshape(prediction_shape)

    # Plot the predicted labels
    fig = go.Figure()

    # Extract non-zero predicted labels for visualization
    non_zero_indices = np.nonzero(reshaped_predictions)
    x, y, z = non_zero_indices
    colors = reshaped_predictions[non_zero_indices]

    fig.add_trace(go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='markers',
        marker=dict(
            size=2,
            color=colors,
            colorscale='Viridis',
            opacity=0.5
        )
    ))

    fig.update_layout(
        title='3D Scatter Plot of Predicted Labels',
        scene=dict(
            xaxis_title='X Axis',
            yaxis_title='Y Axis',
            zaxis_title='Z Axis'
        )
    )

    # Display the plot
    fig.show()


In [69]:
test("NIFTY/CVD0227_Image.nii.gz")

Loading NIFTI file from NIFTY/CVD0227_Image.nii.gz
Normalizing data
Data loaded and preprocessed
Processed data shape: (512, 512, 330)
Downsampled data shape: (512, 512, 330)
Starting HOG3D computation
Processed 10000/10664100 blocks
Processed 20000/10664100 blocks


KeyboardInterrupt: 